In [1]:
from pydantic import BaseModel, Field
from typing import List
import instructor
from litellm import completion

from openai import OpenAI
from langchain_core.messages import AIMessage, ToolMessage

import json

from typing import List, Dict, Any, Annotated, Optional
from operator import add
from jinja2 import Template

In [2]:
# Pydantic Models

class ToolCall(BaseModel):
    name: str
    arguments: dict
    server: str

# from typing import Optional

# class ToolCall(BaseModel):
#     name: str
#     arguments: dict = Field(alias="parameters")
#     server: Optional[str] = None


class RAGUsedContext(BaseModel):
    id: int
    description: str

class AgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)
    final_answer: bool = Field(default=False)
    retrieved_context_ids: List[RAGUsedContext]

class QAAgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)
    final_answer: bool = Field(default=False)
    retrieved_context_ids: List[RAGUsedContext]

class ClassifierAgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)
    final_answer: bool = Field(default=False)


class Delegation(BaseModel):
    agent: str
    task: str = Field(default="")

class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: list[Delegation]
    final_answer: bool = Field(default=False)
    answer: str


class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    answer: str = ""
    iteration: int = Field(default=0)
    classifier_iteration: int = Field(default=0)
    coordinator_iteration: int = Field(default=0)
    job_posting_qa_final_answer: bool = Field(default=False)
    classifier_final_answer: bool = Field(default=False)
    coordinator_final_answer: bool = Field(default=False)
    qa_available_tools: List[Dict[str, Any]] = []
    classifier_available_tools: List[Dict[str, Any]] = []
    qa_tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    classifier_tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    retrieved_context_ids: List[RAGUsedContext] = []
    # NEW fields for classifier integration
    # retrieved_job_posting: Optional[str] = ""            # stores the retrieved job posting text
    classification_result: str = ""  # store fraud classification result
    user_intent: str = ""
    plan: list[Delegation] = Field(default_factory=list)
    next_agent: str = ""

In [3]:
def lc_messages_to_regular_messages(msg):

    if isinstance(msg, dict):
        
        if msg.get("role") == "user":
            return {"role": "user", "content": msg["content"]}
        elif msg.get("role") == "assistant":
            return {"role": "assistant", "content": msg["content"]}
        elif msg.get("role") == "tool":
            return {
                "role": "tool", 
                "content": msg["content"], 
                "tool_call_id": msg.get("tool_call_id")
            }
        
    elif isinstance(msg, AIMessage):

        result = {
            "role": "assistant",
            "content": msg.content
        }
        
        if hasattr(msg, 'tool_calls') and msg.tool_calls and len(msg.tool_calls) > 0 and not msg.tool_calls[0].get("name").startswith("functions."):
            result["tool_calls"] = [
                {
                    "id": tc["id"],
                    "type": "function",
                    "function": {
                        "name": tc["name"].replace("functions.", ""),
                        "arguments": json.dumps(tc["args"])
                    }
                }
                for tc in msg.tool_calls
            ]
            
        return result
    
    elif isinstance(msg, ToolMessage):

        return {"role": "tool", "content": msg.content, "tool_call_id": msg.tool_call_id}
    
    else:

        return {"role": "user", "content": str(msg)}

In [4]:
def coordinator_agent_node(state, models = ["groq/llama-3.1-8b-instant","gpt-4.1"]) -> dict:

    prompt_template = """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- job_posting_qa_agent: The user is asking a question about a job posting. This can be a question about its specifications, descriptions, text entities etc.
- classifier_agent: The user is asking to classifiy job postings on whether the posting is real or not.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true

Additional instructions:

- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

    prompt = Template(prompt_template).render()

    messages = state.messages

    conversation = []

    for msg in messages:
        conversation.append(lc_messages_to_regular_messages(msg))

    client = instructor.from_openai(OpenAI())

    for model in models:
        try:
            response, raw_response = client.chat.completions.create_with_completion(
                model=model,
                response_model=CoordinatorAgentResponse,
                messages=[{"role": "system", "content": prompt}, *conversation],
                temperature=0,
            )
            break
        except Exception as e:
            print(f"Error with model {model}: {e}")
            continue


    if response.final_answer:
        ai_message = [AIMessage(
            content=response.answer,
        )]
    else:
        ai_message = []

    return {
        "messages": ai_message,
        "answer": response.answer,
        "next_agent": response.next_agent,
        "plan": response.plan,
        "coordinator_final_answer": response.final_answer,
        "coordinator_iteration": state.coordinator_iteration + 1,
        "trace_id": ""
    }

In [8]:
initial_state = State(messages=[{"role": "user", "content": "What is the weather today?"}])

In [9]:
answer = coordinator_agent_node(initial_state)

Error with model groq/llama-3.1-8b-instant: Error code: 400 - {'error': {'message': 'invalid model ID', 'type': 'invalid_request_error', 'param': None, 'code': None}}


In [66]:
answer

{'messages': [AIMessage(content="I'm sorry, but I can only assist with questions related to job postings, such as their details or authenticity. I am unable to provide weather information.", additional_kwargs={}, response_metadata={})],
 'answer': "I'm sorry, but I can only assist with questions related to job postings, such as their details or authenticity. I am unable to provide weather information.",
 'next_agent': '',
 'plan': [],
 'coordinator_final_answer': True,
 'coordinator_iteration': 1,
 'trace_id': ''}

In [8]:
coordinator_eval_dataset = [
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Whats is the weather today?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What is job 10397 about?"}
            ]
        },
        "outputs": {
            "next_agent": "job_posting_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you classify a posting with ID 10397 and tell me if it's real or fake?"}
            ]
        },
        "outputs": {
            "next_agent": "classifier_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you classify job 123 and also job 456 and tell me if they are real or fake?"}
            ]
        },
        "outputs": {
            "next_agent": "classifier_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Is job 123 a fraudulent job posting and explain why?"}
            ]
        },
        "outputs": {
            "next_agent": "classifier_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What of kind things can you classify?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you help me with my request?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you find me 2 suspicious data job postings?"}
            ]
        },
        "outputs": {
            "next_agent": "job_posting_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Is job 123 related to data analysis?"}
            ]
        },
        "outputs": {
            "next_agent": "job_posting_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What are the entities that can be extracted from job_id 123?"}
            ]
        },
        "outputs": {
            "next_agent": "job_posting_qa_agent",
            "coordinator_final_answer": False
        }
    }
]

# Upload dataset to LangSmith

In [9]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

dataset_name = "coordinator-evaluation-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating routing of the coordinator agent"
)

In [10]:
for item in coordinator_eval_dataset:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"messages": item["inputs"]["messages"]},
        outputs={
            "next_agent": item["outputs"]["next_agent"],
            "coordinator_final_answer": item["outputs"]["coordinator_final_answer"]
        }
    )

# Run Evaluation

In [15]:
from langsmith import Client
import os

ls_client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

In [16]:
def next_agent_evaluator(run, example):

    next_agent_match = run.outputs["next_agent"] == example.outputs["next_agent"]
    final_answer_match = run.outputs["coordinator_final_answer"] == example.outputs["coordinator_final_answer"]

    return next_agent_match and final_answer_match

In [17]:
results = ls_client.evaluate(
    lambda x: coordinator_agent_node(State(messages=x["messages"])),
    data="coordinator-evaluation-dataset",
    evaluators=[
        next_agent_evaluator
    ],
    experiment_prefix="coordinator-evaluation-dataset"
)

/Users/ackl/Projects/ai-engineering-bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'coordinator-evaluation-dataset-64bbdb1b' at:
https://smith.langchain.com/o/cd0c2f03-d905-4ac2-af13-691ee0bcd17d/datasets/fa819a2d-11f0-4086-b60b-2a4165fcf244/compare?selectedSessions=22cbe0f9-93c8-495c-bece-adb2b813727a




8it [00:09,  1.17s/it]


In [18]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
      {
        "role": "user",
        "content": ""
      }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

It seems like you didn't type anything. Please go ahead and ask your question, and I'll do my best to help. I'm here to provide information and answer your queries to the best of my abilities.

In [59]:
import yaml

yaml_file = "../src/api/rag/prompts/classifier_agent.yaml"
prompt_key = 'gpt-4.1'

with open(yaml_file, "r") as f:
    config = yaml.safe_load(f)

template_content = config["prompts"][prompt_key]

template = Template(template_content)
template.render()

'You are part of a Analyst Assistant that can classify whether a job posting is real or fake.\n\nUser may provide either the full job posting text, job title, or a job ID (job_id). \n\nYou will be a conversation history and a list of tools you can use to answer that question.\n\n<Available tools>\n{}\n</Available tools>\nIf a job ID or job title is provided, retrieve the corresponding job posting details using `get_formatted_context` tool and extract top result before classification.\n\nAfter the tools are used you will get the outputs from the tools.\n\nWhen you need to use a tool, format your response as:\n\n<tool_call>\n{"name": "tool_name", "arguments": {...}, "server": {...}}\n</tool_call>\n\nUse names specifically provided in the available tools. Don\'t add any additional text to the names.\n\nYou should tend to use tools when additional information is needed to answer the question.\n\nIf you set final_answer to True, you should not use any tools.\n\n\nInstructions:\n- Carefully 

In [56]:
config

{'metadata': {'name': 'job_posting_qa_prompt',
  'description': 'This prompt is used for job posting agent.',
  'version': '3.0.0'},
 'prompts': {'gpt-4.1': 'You are part of a Analyst Assistant. The user is a Fraud Analyst and your job is to answer questions about job postings.\n\nYou will be given a question and a list of tools you can use to answer that question.\n\n<Available tools>\n{{ available_tools | tojson }}\n</Available tools>\n\nWhen making tool calls, use this exact format:\n{\n    "name": "tool_name",\n    "arguments": {\n        "parameter1": "value1",\n        "parameter2": "value2",\n    },\n    "server": "server_url"\n}\n\nCRITICAL: All parameters must go inside the "arguments" object, not at the top level of the tool call.\n\nExamples:\n- Get formatted item context:\n{\n    "name": "get_formatted_item_context",\n    "arguments": {\n        "query": "Kool kids toys.",\n        "top_k": 5\n    },\n    "server": "http://items_mcp_server:8000/mcp"\n}\n\n- Get formatted us